In [2]:
import sys
sys.path.append("..")

from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pickle
import yaml
import pandas as pd

from lib.notebooks import plot_groups_metrics
from quantizer import Quantizer

In [7]:
quantizers_path = glob("../out/quantizer/*/")
quantizers_path.sort()

In [13]:
groups_metrics = {}
quantizers_loss = {
    "dataset": [],
    "hidden_layers": [],
    "dropout_p": [],
    "learning_rate": [],
    "final_loss": [],
}

for quantizer_path in quantizers_path:
    quantizer = Quantizer.reload(synthesizer_path, load_nn=False)
    config = quantizer.config
    with open("%s/metrics.pickle" % quantizer_path, "rb") as f:
        metrics = pickle.load(f)
    final_loss = min(metrics["validation"]["total_loss"])
    
    # quantizers_loss["dataset"].append(config['dataset']['name'])
    # quantizers_loss["hidden_layers"].append(f"{len(config['model']['hidden_layers'])}x{config['model']['hidden_layers'][0]}")
    # quantizers_loss["dropout_p"].append(config['model']['dropout_p'])
    # quantizers_loss["learning_rate"].append(config['training']['learning_rate'])
    # quantizers_loss["final_loss"].append(final_loss)
    
    group_name = "\n".join((
        f"{','.join(config['dataset']['names'])}",
        f"hidden_layers={len(config['model']['hidden_dims'])}x{config['model']['hidden_dims'][0]}",
    ))
    
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][quantizer_path] = metrics

# quantizers_loss = pd.DataFrame(quantizers_loss)

In [17]:
metrics_name = [
    "total_loss",
    "reconstruction_error",
    "vq_loss",
]

def show_metrics(split_name="validation"):
    plot_groups_metrics(groups_metrics, metrics_name, split_name)
display(ipw.interactive(show_metrics, split_name=["train", "validation"]))

interactive(children=(Dropdown(description='split_name', index=1, options=('train', 'validation'), value='vali…